In [5]:
%%writefile laminarflow/_cruisecontrol.py
import tensorflow as tf
import pickle as pkl
    
class _tf_temp():
    def __init__(self, name):
        self.name = name
class _method_temp():
    def __init__(self, name, method_name):
        self.name = name
        self.method_name = method_name

class CruiseControl():
    """
    Laminar Flow's Cruise Control method automates several
    time saving tasks for Tensorflow, allowing for quicker
    prototyping and testing.
    
    Among these abilities is automatic saving in an encapsulated
    tensorflow session. This makes it so you don't need to
    keep open a session to keep the values. However, it does
    require you to use `tf.get_variable` to define variables
    instead of `tf.Variable` directly.
    
    On top of that, automatic initialization of uninitialized
    variables allows for this structure to be dynamically updated
    and have no cat and mouse hunt for errors.
    
    Pickling and unpickling has been implemented in this class
    under the strict conditions that all non-variable tensors
    used in args to the `add` function are the result of previous
    functions input to the `add` function and all `function`s used
    have to either be functions or methods bound to results of
    previous calls to add. That is, when calling `add`, only use
    direct attributes of that CruiseControl instance or variables
    controlled by it and only use functions or methods controlled
    by that instance as well.
    
    Since TensorFlow objects are not pickleable directly, args
    and kwargs to `add` have to be easily sanitized, unless you
    want to do it yourself. More complex sanitization is possible,
    if somewhat difficult. To sanitize yourself, currently you must
    create the magic functions in whatever you pass to `add` if
    it isn't automatically taken care of already.
    
    More documentation to follow. This all needs to be reworded
    to make more sense.
    """
#Constructor
    def __init__(self, save_file_name, unique_identifier = None):
        #collect_variables()
        self._vars = set()
        self._var_pkl = list()
        self._file_name = save_file_name
        self._uuid = unique_identifier if unique_identifier else hex(id(self))[2:]
        self._g = tf.Graph()
        with self._g.as_default():
            while True:
                try:
                    with tf.variable_scope(self._uuid):
                        with tf.variable_scope(self._uuid):
                            tf.get_variable("initialized",shape=[1])
                    break
                except:
                    if unique_identifier is not None:
                        raise NameError("Error: {} is an invalid unique identifier.".format(self._uuid))
                    self._uuid += "0"
        self._sess = None
        self._opened = 0
#Structure
    def add(self, name, function, *args, **kwargs):
        self._sess = None
        if hasattr(self, name):
            raise AttributeError("Error: {} already defined.".format(name))
        current = set(self._g.get_collection("variables"))
        #sanitize
        sanitized_args = [self.sanitize(arg) for arg in args]
        sanitized_kwargs = {key:self.sanitize(value) for key,value in kwargs.items()}
        sanitized_func = self.sanitize(function)
        #Test.
        try:
            pkl.dumps([sanitized_func, sanitized_args, sanitized_kwargs])
        except:
            raise ValueError("Error: Unable to sanitize.")
        #unsanitize
        unsanitized_func = self.unsanitize(function)
        unsanitized_args = [self.unsanitize(arg) for arg in args]
        unsanitized_kwargs = {key:self.unsanitize(value) for key,value in kwargs.items()}
        with self._g.as_default():
            with tf.variable_scope(self._uuid):
                with tf.variable_scope(name):
                    setattr(self, name, unsanitized_func(*unsanitized_args, **unsanitized_kwargs))
        self._vars |= set(self._g.get_collection("variables")) - current
        self._var_pkl.append([name, sanitized_func, sanitized_args, sanitized_kwargs])
        if isinstance(getattr(self, name), tf.train.Optimizer):
            #Initialize slots?
            pass
        return self
    def last_added(self):
        try:
            return getattr(self, self._var_pkl[-1][0])
        except:
            return None
#Sanitization
    def removeUUIDandColon(self, name):
        #start = 3 #len("{0}")
        name = name[len(self._uuid):].replace(self._uuid, "{0}")
        #restart = var.name[start:].find('/') + start + 1
        #restart = name.find('/') + 1
        #return name[restart:name.rfind(":")]
        return name[name.find('/') + 1:name.rfind(":")]
    def sanitize(self, obj):
        if hasattr(obj, "__self__"):
            method_self = getattr(obj, "__self__")
            for name,_,_,_ in self._var_pkl:
                if getattr(self, name) is method_self:
                    return _method_temp(name, obj.__name__)
            for var in self._var:
                if var is method_self:
                    return _method_temp(self.removeUUIDandColon(var.name), obj.__name__)
        try:
            pkl.dumps(obj)
            return obj
        except:
            return _tf_temp(self.removeUUIDandColon(obj.name))
    def unsanitize(self, obj):
        if isinstance(obj, _tf_temp):
            try:
                return tf.get_variable(obj.name.format(self._uuid))
            except:
                end = obj.name.find('/')
                return getattr(self, obj.name[:end])
        if isinstance(obj, _method_temp):
            try:
                return getattr(tf.get_variable(obj.name.format(self._uuid)), obj.method_name)
            except:
                return getattr(getattr(self, obj.name.format(self._uuid)), obj.method_name)
        return obj
#Features
    def set_file(self, save_file_name):
        self._file_name = save_file_name
    setFile = set_file
#Values
    def save(self, save_file_name = None):
        variables = []
        for i in self._vars:
            if tf.is_variable_initialized(i).eval():
                try:
                    variables.append((self.removeUUIDandColon(i.name),i.value().eval()))
                except:
                    #TODO: Don't do this. Limit exceptions to known expected ones.
                    pass
        if save_file_name is None:
            save_file_name = self._file_name
        with open(self._file_name, "wb") as file:
            pkl.dump(variables, file)
    def load(self, save_file_name = None):
        """
        Must be done from within a with block.
        """
        try:
            if save_file_name is None:
                save_file_name = self._file_name
            with open(self._file_name, "rb") as file:
                variables = pkl.load(file)
        except:
            #TODO: Don't do this. Limit exceptions to known expected ones.
            #print("Unable to load pkl file.")
            return
        with tf.variable_scope(self._uuid, reuse=True):
            for i in variables:
                try:
                    tf.get_variable(i[0].format(self._uuid)).assign(i[1]).eval(session=self._sess)
                except ValueError as msg:
                    #print(str(msg))
                    pass
    def transfer_from(self, save_file_name):
        self.load(save_file_name)
        #yes, this is just an alias for readability sake, and to force a file name.
#Serialization
    def __reduce__(self):
        return (CruiseControl, (self._file_name,), self._var_pkl)
    def __setstate__(self, state):
        for i in state:
            self.add(i[0],i[1],*i[2],**i[3])
        return self
    '''
#Initialization
    def initialize_variables(self, specifically=None):
        uninitialized = []
        start = len(self._uuid)
        with tf.variable_scope(self._uuid, reuse=True):
            for name in self._sess.run(tf.report_uninitialized_variables(self._vars)):
                name = name.decode("utf-8")
                restart = name[start:].find('/') + start + 1
                end = name.rfind(":")
                if end == -1:
                    end = None
                print(name)
                print(restart)
                print(end)
                print(name[restart:end])
                uninitialized.append(tf.get_variable(name[restart:end]))
        self._sess.run(tf.initialize_variables(uninitialized))
    '''
#Functionality
    @property
    def sess(self):
        if self._opened:
            return self._sess
        with self._g.as_default():
            self._init = tf.global_variables_initializer()
        self._sess = tf.Session(graph=self._g)
        return self._sess
    def __enter__(self):
        sess = self.sess
        self._opened += 1
        if self._opened == 1:
            sess.__enter__()
            #self.initialize_variables()
            # trying to be smart about initialization seems
            #  to be a bad idea for some reason?
            self._sess.run(self._init)
            # We're just going to load values back anyway.
            self.load()
            return self
        else:
            return self
    def __exit__(self, *args):
        self.save()
        self._opened -= 1
        if self._opened == 0:
            return self._sess.__exit__(*args)
    def run(self, *args, **kwargs):
        """
        This provides Session.run access to the CruiseControlled sesion.
        """
        with self:
            return self._sess.run(*args, **kwargs)


Overwriting laminarflow/_cruisecontrol.py


In [5]:
%%writefile laminarflow/_filedescriptors.py
from laminarflow import CruiseControl

# These will basically just read the file as said type,
#  and the result will be dictlike already, so we can
#  just pass it in appropriately.

layer_types = {}

def loadDictlike(dct):
    """
    Given a network description as a list of dictionaries,
    constructs a CruiseControl wrapped network with the
    given definition.
    
    The values are translated using function pointers in
    layer_types, which can be modified directly to give
    new layer support. Helper functions will be provided
    in future versions to speed up the process.
    
    The naming scheme of layers and description parameters
    is inspired by Caffe's prototxt files in hopes of being
    able to directly load Caffe's Modelzoo files, with
    trained parameters and all.
    """
    pass

def loadJSON(filename):
    pass

def loadYAML(filename):
    pass


Overwriting laminarflow/_filedescriptors.py


In [6]:
%%writefile laminarflow/__init__.py
from laminarflow._cruisecontrol import CruiseControl
#from laminarflow._filedescriptors import loadDictlike
#from laminarflow._filedescriptors import loadJSON
#from laminarflow._filedescriptors import loadYAML
#from laminarflow._filedescriptors import layer_types


Overwriting laminarflow/__init__.py


In [3]:
%%writefile setup.py
import os
from setuptools import setup

# Utility function to read the README file.
# Used for the long_description.  It's nice, because now 1) we have a top level
# README file and 2) it's easier to type in the README file than to put a raw
# string in below ...
def read(fname):
    return open(os.path.join(os.path.dirname(__file__), fname)).read()

setup(
    name = "LaminarFlow",
    version = "1.3.0.0",
    author = "Chris Dusold",
    author_email = "LaminarFlow@ChrisDusold.com",
    description = ("A meta class to wrap and automate TensorFlow."),
    license = read("LICENSE"),
    keywords = "TensorFlow",
    #url = "http://laminarflow.rtfd.org/",
    packages=['laminarflow'],
    long_description=read('README.md'),
    install_requires=['pyyaml'],
    classifiers=[
        "Development Status :: 4 - Beta",
        "Intended Audience :: Developers",
        "Intended Audience :: Science/Research",
        "License :: OSI Approved :: MIT License",
        "Natural Language :: English",
        "Operating System :: OS Independent", #Hopefully.
        "Programming Language :: Python",
        "Topic :: Scientific/Engineering",
        "Topic :: Scientific/Engineering :: Artificial Intelligence",
        "Topic :: Utilities",
    ],
)


Overwriting setup.py


In [1]:
import tensorflow as tf

def clu(x, outdim, indim=None):
    if indim:
        try:
            indim = list(indim)
        except:
            indim = [indim]
    else:
        indim = x.get_shape()[1:].as_list()
    try:
        outdim = list(outdim)
    except:
        outdim = [outdim]
    C = tf.get_variable("C", initializer=tf.truncated_normal(indim+outdim, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(outdim, stddev=0.1))
    cluKernel = -tf.reduce_sum(tf.abs(tf.sub(tf.expand_dims(x,len(indim)+1),
                                             tf.expand_dims(C,0))),
                               1) + b
    cluKernel._C = C
    cluKernel._b = b
    return cluKernel

def lin(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.truncated_normal(shape[1:], stddev=0.1))
    linKernel = tf.matmul(x,W) + b
    linKernel._W = W
    linKernel._b = b
    return linKernel

#Convolution and Maxpool shortcuts
def conv2d(x, shape):
    W = tf.get_variable("W", initializer=tf.truncated_normal(shape, stddev=0.1))
    b = tf.get_variable("b", initializer=tf.constant(0.1, shape=shape[-1:]))
    convKernel = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME') + b
    convKernel._W = W
    convKernel._b = b
    return convKernel

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def cross_entropy(y,yhat):
    return -tf.reduce_sum(y*tf.log(yhat + 1e-9)) #Without epsilon, it crashes.

def accuracy_test(y,yhat):
    correct_prediction = tf.equal(tf.argmax(yhat,1), tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [2]:
from laminarflow import CruiseControl

test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
#test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
#test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
#test.add('loss', cross_entropy, test.expected_output, test.output)
#test.add('train_step', test.optim.minimize, test.loss)

add
{<tensorflow.python.ops.variables.Variable object at 0x7f5a1bdfec18>}
add
{<tensorflow.python.ops.variables.Variable object at 0x7f5a1bdfec18>}
add
{<tensorflow.python.ops.variables.Variable object at 0x7f5a1bdfec18>, <tensorflow.python.ops.variables.Variable object at 0x7f5a1b5954a8>, <tensorflow.python.ops.variables.Variable object at 0x7f59f55700f0>}


In [3]:
import numpy as np

with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))

enter
sess
1
load
enter
sess
2
exit
save
{<tensorflow.python.ops.variables.Variable object at 0x7f5a1b5954a8>, <tensorflow.python.ops.variables.Variable object at 0x7f59f55700f0>}
Tensor("7f5a1bdeddd8/h/b/read:0", shape=(10,), dtype=float32)
Tensor("7f5a1bdeddd8/h/W/read:0", shape=(784, 10), dtype=float32)
1
[[  2.01178156e-02   1.76191106e-01   2.41327912e-01   1.83001370e-03
    1.02801770e-02   7.52868014e-04   2.48486991e-04   3.14141423e-01
    2.17452794e-01   1.76572651e-02]]
[[  2.01178156e-02   1.76191106e-01   2.41327912e-01   1.83001370e-03
    1.02801770e-02   7.52868014e-04   2.48486991e-04   3.14141423e-01
    2.17452794e-01   1.76572651e-02]]
exit
save
{<tensorflow.python.ops.variables.Variable object at 0x7f5a1b5954a8>, <tensorflow.python.ops.variables.Variable object at 0x7f59f55700f0>}
Tensor("7f5a1bdeddd8/h/b/read:0", shape=(10,), dtype=float32)
Tensor("7f5a1bdeddd8/h/W/read:0", shape=(784, 10), dtype=float32)
0
enter
sess
1
load
enter
sess
2
exit
save
{<tensorflow.p

In [4]:
import pickle as pkl

test2 = pkl.loads(pkl.dumps(test))
with test2 as sess:
    print(test2.output.eval({test2.input:np.ones([1,784])}))

add
{<tensorflow.python.ops.variables.Variable object at 0x7f59f57e7c50>}
add
{<tensorflow.python.ops.variables.Variable object at 0x7f59f57e7c50>}
add
{<tensorflow.python.ops.variables.Variable object at 0x7f59ec187da0>, <tensorflow.python.ops.variables.Variable object at 0x7f59ec17f7b8>, <tensorflow.python.ops.variables.Variable object at 0x7f59f57e7c50>}
enter
sess
1
load
enter
sess
2
exit
save
{<tensorflow.python.ops.variables.Variable object at 0x7f59ec187da0>, <tensorflow.python.ops.variables.Variable object at 0x7f59ec17f7b8>}
Tensor("7f5a1b5bd9b0/h/b/read:0", shape=(10,), dtype=float32)
Tensor("7f5a1b5bd9b0/h/W/read:0", shape=(784, 10), dtype=float32)
1
[[  2.01178156e-02   1.76191106e-01   2.41327912e-01   1.83001370e-03
    1.02801770e-02   7.52868014e-04   2.48486991e-04   3.14141423e-01
    2.17452794e-01   1.76572651e-02]]
exit
save
{<tensorflow.python.ops.variables.Variable object at 0x7f59ec187da0>, <tensorflow.python.ops.variables.Variable object at 0x7f59ec17f7b8>}
Ten

In [29]:
test = CruiseControl("test")
test.add('input',tf.placeholder, "float", shape=[None, 784])
test.add('expected_output', tf.placeholder, "float", shape=[None, 10])
test.add('h', lin, test.input,[784,10])
test.add('output', tf.nn.softmax, test.h)
test.add('optim', tf.train.AdadeltaOptimizer, 1e-4)
test.add('loss', cross_entropy, test.expected_output, test.output)
test.add('train_step', test.optim.minimize, test.loss)
#testing
test._initialized = set(x for x in test._vars)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
print(test.run(test.output,feed_dict={test.input:np.ones([1,784])}))

[[  3.91621422e-03   2.82290950e-02   2.67429184e-02   8.02965078e-04
    4.82511277e-05   9.30676281e-01   7.31190317e-04   1.91889732e-04
    8.63836240e-03   2.26531192e-05]]
[[  1.39723779e-04   5.80451684e-03   2.04441160e-01   1.72922730e-01
    3.24747467e-04   5.88755645e-02   2.04511788e-02   8.05121437e-02
    1.72266200e-01   2.84262031e-01]]


In [30]:
print(test.optim._zeros_slot(test.loss, 'accum_update', test.optim._name))

Tensor("7f956810a0b8_6/loss/Neg/Adadelta/read:0", shape=(), dtype=float32)


In [31]:
test.optim.get_slot(test.loss, 'accum_update').name

'7f956810a0b8_6/loss/Neg/Adadelta:0'

In [32]:
print([i.name for i in tf.trainable_variables()])

['test2/W:0']


In [33]:
test._var_pkl[6][1].method_name

'minimize'

In [34]:
import pickle as pkl

test2 = pkl.loads(pkl.dumps(test))
with test2 as sess:
    print(test2.output.eval({test2.input:np.ones([1,784])}))

[[  1.69830758e-03   4.72098218e-05   5.72629273e-01   2.15697964e-03
    5.06564493e-06   2.91877985e-02   2.15289459e-01   9.44951400e-02
    8.30261037e-02   1.46463246e-03]]


In [35]:
test.add('relu', tf.nn.relu, test.h)
test.add('h2', lin, test.relu, [10,10])
test.add('output2', tf.nn.softmax, test.h2)
test.add('loss2', cross_entropy, test.expected_output, test.output2)
test.add('train_step2', test.optim.minimize, test.loss2)
with test as sess:
    print(test.output.eval({test.input:np.ones([1,784])}))
with test as sess:
    print(test.output2.eval({test.input:np.ones([1,784])}))

[[  1.56685505e-02   8.00773734e-04   6.37518406e-01   4.62362124e-03
    1.42823666e-01   1.12912012e-03   2.60556606e-03   2.78750435e-04
    1.93829164e-01   7.22434314e-04]]
[[ 0.08158474  0.12712577  0.09410271  0.06918326  0.1128696   0.11112418
   0.08440532  0.11400516  0.14154947  0.06404966]]


In [14]:
test.optim._get_or_create_slot(test.loss2,'accum')

AttributeError: 'AdadeltaOptimizer' object has no attribute '_get_or_create_slot'

In [15]:
tf.is_variable_initialized(t).eval()

NameError: name 't' is not defined

In [16]:
import tensorflow as tf

In [17]:
with tf.variable_scope("test2"):
    test = {}
    test['in'] = tf.placeholder("float", shape=[1,2])
    test['out'] = tf.placeholder("float", shape=[1,3])
    test['W'] = tf.get_variable("W", shape=[2,3])
    test['loss'] = tf.nn.l2_loss(tf.matmul(test['in'], test['W']) - test['out'])
    test['optim'] = tf.train.AdadeltaOptimizer()
    test['train_step'] = test['optim'].minimize(test['loss'])

In [18]:
test['train_step'].name

'test2/Adadelta'

In [20]:
tf.global_variables()

In [21]:
[i.name for i in tf.global_variables()]

['7f95781c5908_6/loss/Neg/Adadelta:0',
 'test2/W:0',
 'test2/test2/W/Adadelta:0',
 'test2/test2/W/Adadelta_1:0']

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
with tf.variable_scope("test2", reuse=True):
    W = tf.get_variable("W")
    #WAda = tf.get_variable("test2/W/Adadelta:0")
    with tf.variable_scope("test2", reuse=True):
        with tf.variable_scope("W", reuse=True):
            WAda1 = tf.get_variable("Adadelta_1:0")
            #Variable test2/test2/W/Adadelta_1:0 [...] was not created with tf.get_variable()

ValueError: Variable test2/test2/W/Adadelta_1:0 does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

In [25]:
"{0} is {0}".format("this")

'this is this'

In [26]:
len("{0}")

3

In [36]:
test.loss._shape_as_list()

[]

In [37]:
type(test.output)

tensorflow.python.framework.ops.Tensor

In [38]:
set(dir(test.input))

{'OVERLOADABLE_OPERATORS',
 '__abs__',
 '__add__',
 '__and__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__invert__',
 '__iter__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_add_consumer',
 '_as_node_def_input',
 '_consumers',
 '_dtype',
 '_handle_dtype',
 '_handle_shape',
 '_op',
 '_override_operator',
 '_shape',
 '_shape_as_list',
 '_value_index',
 'consumers',
 

In [39]:
[x for x in test.output.op.inputs]

[<tf.Tensor '7f956810a0b8_3/h/add:0' shape=(?, 10) dtype=float32>]

In [40]:
dir(test.output.graph)

['_ControlDependenciesController',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_function',
 '_add_op',
 '_apply_device_functions',
 '_as_graph_def',
 '_as_graph_element_locked',
 '_attr_scope',
 '_attr_scope_map',
 '_building_function',
 '_check_not_finalized',
 '_collections',
 '_colocation_stack',
 '_container',
 '_control_dependencies_for_inputs',
 '_control_dependencies_stack',
 '_control_flow_context',
 '_current_control_dependencies',
 '_default_original_op',
 '_device_function_stack',
 '_finalized',
 '_functions',
 '_get_control_flow_context',
 '_get_function',
 '_gradient_override_map',
 '_graph_def_versions',
 '_handle_deleters',
 '_handle_feeders',
 '_ha

In [41]:
test.output.graph.get_tensor_by_name(test.input.name)

<tf.Tensor '7f956810a0b8_1/input/Placeholder:0' shape=(?, 784) dtype=float32>

In [42]:
test.output.graph.is_feedable(test.output)

True

In [43]:
[i.name for i in test._vars]

[]

In [44]:
tf.is_variable_initialized(test.input)

TypeError: Input 'ref' of 'IsVariableInitialized' Op requires l-value input

In [45]:
test._sess.graph

In [46]:
sess = tf.Session()

In [47]:
with sess:
    test.expected_output.eval(feed_dict={test.expected_output:np.ones([1,10])})

ValueError: Cannot use the default session to evaluate tensor: the tensor's graph is different from the session's graph. Pass an explicit session to `eval(session=sess)`.

In [48]:
tf.Graph().get_tensor_by_name(test.input.name)

KeyError: "The name '7f956810a0b8_1/input/Placeholder:0' refers to a Tensor which does not exist. The operation, '7f956810a0b8_1/input/Placeholder', does not exist in the graph."

In [49]:
tf.get_default_graph().get_tensor_by_name(test.h.name)

KeyError: "The name '7f956810a0b8_3/h/add:0' refers to a Tensor which does not exist. The operation, '7f956810a0b8_3/h/add', does not exist in the graph."

In [50]:
g = tf.get_default_graph()

In [51]:
g.unique_name(test.input)

<tf.Tensor '7f956810a0b8_1/input/Placeholder:0' shape=(?, 784) dtype=float32>

In [52]:
g = tf.Graph()
g.as_default()

In [53]:
with g.as_default():
    a = tf.placeholder('float',shape=[None,10])
    b = tf.placeholder('float',shape=[10,1])
    out = a*b

In [54]:
with tf.Session(graph=g):
    print(out.eval(feed_dict={a:np.ones([10,10]),b:[[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]}))

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]
